# Accessing COS-Halos from `pyigm`


In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.interpolate import interp1d 

from astropy.io import fits
from astropy.io import ascii

## 1. Loading COS-Halos

Load in the COS-Halos database from pyigm. This takes ~1 min.

In [5]:
# imports
import pdb
from pyigm.cgm.cos_halos import COSHalos
from pyigm.cgm import cos_halos as pch

reload(pch)
cos_halos = pch.COSHalos()
cos_halos.load_mega()

/Users/howk/anaconda/lib/python2.7/site-packages/linetools-0.2.dev1603-py2.7.egg/linetools/lists/linelist.py:394: RuntimeWarning: divide by zero encountered in log10
  self._data['log(w*f)'] = np.log10(qm_strength)
/Users/howk/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:867: RuntimeWarning: invalid value encountered in less_equal
  return umath.less_equal(x, self.critical_value)


Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/howk/anaconda/lib/python2.7/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)
/Users/howk/anaconda/lib/python2.7/site-packages/linetools-0.2.dev1603-py2.7.egg/linetools/analysis/linelimits.py:165: UserWarning: Redshift=0.  If this is unexpected, set _z and reset limits
  warnings.warn("Redshift=0.  If this is unexpected, set _z and reset limits")


Be patient, using /Users/howk/anaconda/lib/python2.7/site-packages/pyigm-0.1.dev553-py2.7.egg/pyigm/data/CGM/COS_Halos/cos-halos_systems.v1.1.tar.gz to load
Skipping a likely folder: CGM_JSON


/Users/howk/anaconda/lib/python2.7/site-packages/pyigm-0.1.dev553-py2.7.egg/pyigm/cgm/cos_halos.py:264: UserWarning: This method will be DEPRECATED
  warnings.warn("This method will be DEPRECATED")


## 2. Accessing metal ion columns
It seems the following is a path to accessing the adopted (?) metal ion columns.

In [6]:
ions = cos_halos.ion_tbl
SiII = [14,2] #[Z_metal, ionic state]
SiIItable = ions(SiII,fill_ion=True)

In [7]:
SiIItable.colnames

['name',
 'Z',
 'ion',
 'A',
 'Ej',
 'z',
 'vmin',
 'vmax',
 'flag_N',
 'logN',
 'sig_logN',
 'cgm_name']

In [8]:
gg = np.where(SiIItable['flag_N'] > 0.005)
SiIItable[gg]['name','z','logN','sig_logN','flag_N','cgm_name']

name,z,logN,sig_logN,flag_N,cgm_name
unicode32,float64,float64,float64,int64,unicode17
J022614.5+001529.8_z0.227,0.227439895272,12.8962248284,9.0,3,J0226+0015_268_22
J040149-054056.6_z0.220,0.21968537569,12.4730929207,9.0,3,J0401-0540_67_24
J080359.2+433258.4_z0.253,0.253473222256,12.676755793,9.0,3,J0803+4332_306_20
J091029.8+101413.6_z0.264,0.264117896557,13.1521204431,0.129606187344,1,J0910+1014_242_34
J091029.8+101413.6_z0.143,0.142737910151,12.8889483284,0.0817173346877,1,J0910+1014_34_46
J091440.4+282330.6_z0.244,0.244313806295,12.5580061745,9.0,3,J0914+2823_41_27
J092554.7+400414.1_z0.247,0.247452750802,14.4862854201,1.4032468817e-13,2,J0925+4004_196_22
J092838+602521_z0.154,0.154000431299,14.8278713761,6.39069556969e-14,2,J0928+6025_110_35
J093518.2+020415.5_z0.262,0.262280136347,13.2387255151,9.0,3,J0935+0204_15_28


### Flags:
* 0 = Not observed
* 1 = Measurement
* 2 = Lower limit [*saturated*]
* 3 = Upper limit [*non-detection*]

```

```

### Consider ions of Si II and Si III
Here we will look at the column densities of Si II and Si III, packing them into a 2D table array.

In [9]:
#Access Si II with Z=14, charge=2:
SiII_tbl = cos_halos.ion_tbl([14,2])
#Access Si II with Z=14, charge=3:
SiIII_tbl = cos_halos.ion_tbl([14,3])

#Pack the two ions together:
ion_tbls = [SiII_tbl, SiIII_tbl]

In [10]:
#Look at the output for Si II. Could have printed SiII_tbl instead.
ion_tbls[0]

name,Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN,cgm_name
,,,,1 / cm,,km / s,km / s,,,,
unicode32,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,unicode17
J022614.5+001529.8_z0.227,14,2,0,0.0,0.227439895272,-12.0,332.0,3,12.8962248284,9.0,J0226+0015_268_22
J040149-054056.6_z0.220,14,2,0,0.0,0.21968537569,-131.0,107.0,3,12.4730929207,9.0,J0401-0540_67_24
J080359.2+433258.4_z0.253,14,2,0,0.0,0.253473222256,-112.0,100.0,3,12.676755793,9.0,J0803+4332_306_20
J091029.8+101413.6_z0.264,14,2,0,0.0,0.264117896557,-276.0,25.0,1,13.1521204431,0.129606187344,J0910+1014_242_34
J091029.8+101413.6_z0.143,14,2,0,0.0,0.142737910151,-180.0,39.0,1,12.8889483284,0.0817173346877,J0910+1014_34_46
J091440.4+282330.6_z0.244,14,2,0,0.0,0.244313806295,-118.0,12.0,3,12.5580061745,9.0,J0914+2823_41_27
J092554.7+400414.1_z0.247,14,2,0,0.0,0.247452750802,-105.0,215.0,2,14.4862854201,1.4032468817e-13,J0925+4004_196_22
J092838+602521_z0.154,14,2,0,0.0,0.154000431299,-272.0,104.0,2,14.8278713761,6.39069556969e-14,J0928+6025_110_35


## 3. Accessing H I

In [14]:
NHI = cos_halos.NHI
#sig_NHI = cos_halos.sig_NHI  ##This doesn't work?
flag_NHI = cos_halos.flag_NHI

## Notes on structure of COS-Halos data

* The information in COS-Halos array is split between `cgm_abs` and `galaxy` information.
* The absorption details are stored in `.igm_sys` within the `cgm_abs` bit.
* To access the CGM information best, it seems to help to do `absorber = cos_halos.cgm_abs[j].igm_sys`, where `j` is the specific system.
* Within the `cgm_abs` bit, the individual ions are stored. One can assess which are available using `.list_of_abslines()`.
* Individual abs. lines are accessible, e.g., via `.get_absline('SiIII 1206').attrib`, which has a dict of information.

## Accessing specific transitions

* First we choose a specific sight line. From that we'll access the absorber characteristics.

In [32]:
j = 12
absorber = cos_halos.cgm_abs[j].igm_sys

* **Look at which lines are available.**

In [1]:
xxx = absorber.list_of_abslines()
k = 3
yyy=xxx[k]
yyy

NameError: name 'absorber' is not defined

* **Test for a specific transition.**

In [66]:
#Test for a line
lines = absorber.list_of_abslines()
trans = np.array([iline.name for iline in lines])
aline = (trans == 'SiIII 1206')
print(trans[aline],aline.sum())

#Or
aline = np.where(trans == 'SiIII 1206')
print(trans[aline],aline)

(array(['SiIII 1206'], 
      dtype='|S10'), 1)
(array(['SiIII 1206'], 
      dtype='|S10'), (array([30]),))


* **Test for a transition, then access the column density information if it's available.**

In [75]:
#Test for a line
lines = absorber.list_of_abslines()
trans = np.array([iline.name for iline in lines])
aline = (trans == 'SiIII 1206')
print(trans[aline],aline.sum())
      
#Access a specific line
if (aline.sum() > 0):
    xxx=absorber.get_absline('SiIII 1206').attrib
    print(np.round([xxx['logN'],xxx['sig_logN'],xxx['flag_N']],3))

(array(['SiIII 1206'], 
      dtype='|S10'), 1)
[ 13.741   0.034   2.   ]


## Create arrays to hold H I, Si III columns from COS-Halos

Here we consider another approach to accessing column density information. This requires some looping, as you can't access the information in the list *en masse.* 

Also, we have to make sure to test for Si III, as not all of the absorbers include Si III.

In [85]:
#Create the arrays:
ch_name = []
ch_z = []
ch_rho = []
ch_rvir = []
ch_halomass = []
ch_stellarmass = []
#NHI
ch_NHI = []
ch_sigNHI = []
ch_flagNHI = []
#NSiIII
ch_NSiIII = []
ch_sigNSiIII = []
ch_flagNSiIII = []

In [86]:
#Fill the arrays
for j in np.arange(0,cos_halos.nsys):
    absorber = cos_halos.cgm_abs[j].igm_sys

    #Test for Si III line
    lines = absorber.list_of_abslines()
    trans = np.array([iline.name for iline in lines])
    aline = (trans == 'SiIII 1206')
      
    #Access a specific line
    if (aline.sum() > 0):
        si3 = absorber.get_absline('SiIII 1206').attrib
        ch_NSiIII.append(si3['logN'])
        ch_sigNSiIII.append(si3['sig_logN'])
        ch_flagNSiIII.append(si3['flag_N'])
    else:
        ch_NSiIII.append(0.0)
        ch_sigNSiIII.append(99.)
        ch_flagNSiIII.append(99.)

    
for ch in cos_halos.cgm_abs:     
    ch_name.append(ch.name)
    ch_z.append(ch.z)
    ch_rho.append(ch.rho.value)
 
    ch_NHI.append(ch.NHI)
    #Here we add an array [0,0] to sig_NHI, which ensures
    #  it is a 2-element array.
    sigTemp = ch.sig_NHI+np.zeros(2)
    ch_sigNHI.append(sigTemp)
    if (sigTemp.sum() >= 0.5):
        flagTemp=4
    else:
        flagTemp=ch.flag_NHI
    ch_flagNHI.append(flagTemp)

#### Let's examine the flags:

In [96]:
#for j in np.arange(0,np.size(ch_NSiIII)):                   
#    print([ch_name[j],np.round([ch_NSiIII[j],ch_sigNSiIII[j],ch_flagNSiIII[j]],3)])

for j in np.arange(0,11):                   
    print([ch_name[j],np.round([ch_NSiIII[j],ch_sigNSiIII[j],ch_flagNSiIII[j]],3)])    

[u'J0226+0015_268_22', array([ 12.441,   9.   ,   3.   ])]
[u'J0401-0540_67_24', array([ 12.884,   0.059,   1.   ])]
[u'J0803+4332_306_20', array([ 12.48,   9.  ,   3.  ])]
[u'J0910+1014_242_34', array([  0.,  99.,  99.])]
[u'J0910+1014_34_46', array([ 13.292,   0.073,   2.   ])]
[u'J0914+2823_41_27', array([ 12.639,   0.106,   1.   ])]
[u'J0925+4004_196_22', array([ 13.755,   0.037,   2.   ])]
[u'J0928+6025_110_35', array([ 13.783,   0.042,   2.   ])]
[u'J0935+0204_15_28', array([ 12.769,   9.   ,   3.   ])]
[u'J0943+0531_106_34', array([ 12.893,   0.101,   1.   ])]
[u'J0943+0531_216_61', array([ 12.572,   9.   ,   3.   ])]


*IF* the flags follow Table 3 in Prochaska et al. (2017), we have:
* FLAG = 1: Measurement
* FLAG = 2: Lower limit (saturated)
* FLAG = 3: Upper limit
* FLAG = 99: No observation [jch added]

In [100]:
measure = (np.array(ch_flagNSiIII) == 1.)
measure
#print(ch_NSiIII[measure])
#print(ch_flagNSiIII)

array([False,  True, False, False, False,  True, False, False, False,
        True, False, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False,  True, False,
       False,  True, False, False, False,  True, False, False, False,
        True,  True, False, False, False, False, False, False], dtype=bool)